# Leak Finder Proof of Concept

We don't have access to actual data, so we'll have to take some creative liberties.
Let's start by creating a random 'tree' directed graph with 1 source and many sinks -
somewhat similar to a water distribution network.

In [ ]:
from find_leaks import LeakFinder
from network import RandomWaterDistributionNetwork
import gravis

leak_finder = LeakFinder()
random_network_maker = RandomWaterDistributionNetwork()

In [ ]:
number_of_nodes = 30
flow_input_at_source = 10000
number_of_leaks = 3

random_network = random_network_maker.random_network(
    number_of_nodes,
    flow_input_at_source,
    number_of_leaks
)

figure = gravis.d3(random_network)
figure.display(inline=True)

# Introduce Leaks

At the same time as generating our network, we've also introduced a few leaks in random locations throughout the network
(places where flow measured out of a pipe is less than flow measured into the pipe). Can we find them?

In [ ]:
leaks = leak_finder.find_leaks(random_network)
print(f"Found {len(leaks)} leaks in the network")
print("There are leaks at:")
leaks.sort(key=lambda leak: leak.flow_loss, reverse=True)
for leak in leaks:
    print(f"\t- Node {leak.from_node} with flow loss of ~{leak.flow_loss}")
    for edge in leak.potential_leak_edges():
        print(f"\t\t- Potentially on edge {edge}")

# Check junction's pipes are actually leaking

Our program has provided us with a list of places we could look for the leak; let's double-check
the results are reasonable!

In [ ]:
node = 9
expected = leak_finder.get_flow_at(random_network, node)
actual = leak_finder.get_flow_out_total(random_network, node)

print(f"Expected flow out of node {1} to be {expected}")
print(f"It was actually {actual}")

# Show work orders in webpage frontend for engineers

Finally, we can display our data in priority order, i.e.

- current leaks come first, with more leakage having priority over less leakage
- then we have predictions for future failures based on historical data and temperature trends

In [ ]:
from prettify_work_todo import display_work_todo
display_work_todo(leaks, [])